# **Decoupler-GPU: Accelerated Transcriptional Regulatory Analysis**  

**Author:** [Severin Dicks](https://github.com/Intron7)
**Copyright** [scverse](https://scverse.org)

In this notebook, we explore the updated **Decoupler** functions for inferring transcriptional regulatory activity,  
leveraging the **GPU-accelerated** implementation from **rapids-singlecell**.  

By running these analyses on GPUs, we achieve **faster computation** and **scalability** for large single-cell datasets.  
We will apply **Decoupler** methods to estimate **transcription factor (TF) activity** and **pathway activity**,  
utilizing curated regulatory networks such as **Dorothea** and **PROGENy**.

In [1]:
import rapids_singlecell as rsc
import scanpy as sc
import rmm
import cupy as cp
import decoupler as dc
from rmm.allocators.cupy import rmm_cupy_allocator
rmm.reinitialize(
    managed_memory=False, # Allows oversubscription
    pool_allocator=True, # default is False
)
cp.cuda.set_allocator(rmm_cupy_allocator)

In [2]:
adata = sc.read_h5ad("h5/dli_decoupler.h5ad")

In [3]:
adata = adata.raw.to_adata()

In [4]:
rsc.get.anndata_to_GPU(adata)

### Retrieving Transcription Factor (TF) Regulons with Dorothea  

We start by retrieving the **Dorothea** gene regulatory network, which provides curated transcription factor (TF) target interactions.  
Dorothea assigns confidence levels (`A`, `B`, `C`, `D`, `E`), where `A` represents the highest confidence interactions.  
Here, we filter for **high-confidence** (`A`, `B`, `C`) interactions in human data:  

In [5]:
net = dc.get_dorothea(organism='human', levels=['A','B','C'])

### Transcription Factor Activity Inference with ULM  

To infer transcription factor (TF) activity, we use the **Univariate Linear Model (ULM)** from **Decoupler**.  
ULM estimates the regulatory influence of TFs on gene expression by fitting a linear model for each TF-gene interaction.  

We apply **ULM** using the **Dorothea** regulatory network retrieved earlier. 

In [6]:
%%time
rsc.dcg.run_ulm(mat=adata, net=net, source='source', target='target', weight='weight', verbose=True, use_raw = False, pre_load=True)

Running ulm on mat with 213082 samples and 28065 targets for 296 sources.


  0%|          | 0/22 [00:00<?, ?it/s]

CPU times: user 12.7 s, sys: 222 ms, total: 12.9 s
Wall time: 12.8 s


### Transcription Factor Activity Inference with MLM  

Next, we infer transcription factor (TF) activity using the **Multivariate Linear Model (MLM)** from **Decoupler**.  
Unlike **ULM**, which considers each TF independently, **MLM** accounts for multiple TFs simultaneously,  
providing a more comprehensive estimation of regulatory influences. 

In [7]:
%%time
rsc.dcg.run_mlm(mat=adata, net=net, source='source', target='target', weight='weight', verbose=True, use_raw = False, pre_load=True)

Running mlm on mat with 213082 samples and 28065 targets for 296 sources.


  0%|          | 0/22 [00:00<?, ?it/s]

CPU times: user 13.5 s, sys: 341 ms, total: 13.9 s
Wall time: 13.7 s


### Transcription Factor Activity Inference with AUCell  

Next, we infer transcription factor (TF) activity using **AUCell** from **Decoupler**.  
AUCell estimates TF activity based on the **enrichment of TF target genes within the top-expressed genes** in each cell.  
Unlike regression-based methods (**ULM**, **MLM**), AUCell is a **rank-based approach** that does not assume linear relationships. 

In [8]:
%%time
rsc.dcg.run_aucell(mat=adata, net=net, source='source', target='target', verbose=True, use_raw = False, pre_load=True)

Running aucell on mat with 213082 samples and 28065 targets for 296 sources.


  0%|          | 0/43 [00:00<?, ?it/s]

CPU times: user 34 s, sys: 230 ms, total: 34.3 s
Wall time: 34.1 s


### Retrieving Pathway Activity Models with PROGENy  

To assess pathway activity, we use **PROGENy**, a curated model that links gene expression to pathway activity scores.  
PROGENy provides a **data-driven** approach to infer the activity of major signaling pathways  
by leveraging transcriptional signatures derived from perturbation experiments.  

We retrieve the **top 100 most informative target genes** for each pathway in **human** data  

In [9]:
model = dc.get_progeny(organism="human", top=100)

### Pathway Activity Inference with Weighted Sum (WSUM)  

Next, we infer pathway activity using the **Weighted Sum (WSUM)** method from **Decoupler**.  
WSUM estimates pathway activity by computing a **weighted sum of target gene expression values**,  
where gene weights reflect their importance in the pathway model (e.g., from PROGENy).  

In [10]:
%%time
rsc.dcg.run_wsum(mat=adata, net=model, source='source', target='target', weight='weight', verbose=True, use_raw = False, pre_load=True)

Running wsum on mat with 213082 samples and 28065 targets for 14 sources.


  0%|          | 0/22 [00:00<?, ?it/s]

CPU times: user 39.3 s, sys: 227 ms, total: 39.6 s
Wall time: 39.2 s
